In [1]:
import numpy as np
import pandas as pd
import scipy.stats
import scipy.signal
import matplotlib.pyplot as plt
matplotlib.rcParams['figure.figsize'] = [6.0, 6.0]

NameError: name 'matplotlib' is not defined

# General Setup

We have a target distribution and we will try to sample from it using Metropolis MCMC with various proposal distributions.

Some useful links were found here:  
https://theclevermachine.wordpress.com/2012/11/19/a-gentle-introduction-to-markov-chain-monte-carlo-mcmc/  
https://www.cs.cmu.edu/~epxing/Class/10708-14/scribe_notes/scribe_note_lecture27.pdf  

In [ ]:
# Target Distribution (Stationary Distribution of the Markov Chain)
# Stretched Normal Distribution in 2D
mean = [0, 0]
cov = [[1, 0], [0, 100]]

target_dist = scipy.stats.multivariate_normal(mean, cov)
data = target_dist.rvs(1000)
df = pd.DataFrame(data, columns=["x", "y"])
plt.plot(df.x, df.y, 'o')
plt.axis('equal')
plt.show()

In [ ]:
# Random Walk MCMC Function
def metropolis_step(state, proposal_dist, target_logpdf):
    proposal_dist = proposal_dist(state)
#     proposal_dist = scipy.stats.multivariate_normal(state, [[0.5, 0], [0, 0.5]])
    proposed_state = proposal_dist.rvs(1)
    
    acceptance_prob = min([0, target_dist.logpdf(proposed_state) - target_dist.logpdf(state)])
    
    if np.log(np.random.rand()) < acceptance_prob:
        return proposed_state
    else:
        return state
    
# Acceptance rate at every step
def acceptance_rate_per_step(states):
    transitions = len(states)
    compared_states = np.isclose(np.asarray(states), np.roll(np.asarray(states), 1, axis=0))
    acceptance_rate_per_step = np.asarray([np.sum(compared_states[:i,:])/compared_states[:i,:].shape[0] for 
                                i in range(1, compared_states.shape[0] + 1)])
    return acceptance_rate_per_step 

## Normal, $\sigma$ = 0.5

We procede with a proposal distribtuion which is normal and has a covarience of 0.5 times the identity matrix.

In [ ]:
# Proposal distribution
def proposal_dist_normal_2d(state, covar=[[0.5, 0], [0, 0.5]]):
    return scipy.stats.multivariate_normal(state, covar)

In [ ]:
# "Random Walk" Proposal Distribution Centered at previous state
# Normal Distribution in 2D

mean = [0, 0]
cov = [[0.5, 0], [0, 0.5]]

proposal_dist = scipy.stats.multivariate_normal(mean, cov)
data = proposal_dist.rvs(1000)
df = pd.DataFrame(data, columns=["x", "y"])
plt.plot(df.x, df.y, 'o')
plt.axis('equal')
plt.show()

In [ ]:
# MCMC Execution
states = [np.array([0,0])]
steps = 10000

for i in range(steps):
    states.append(metropolis_step(states[-1], proposal_dist_normal_2d, target_dist.logpdf))
    
states = np.asarray(states)

In [ ]:
# Acceptance rate plot
acps = acceptance_rate_per_step(states)
plt.figure(figsize=(8, 4))
plt.plot(range(steps + 1), acps)
plt.xlabel('Steps')
plt.ylabel('Acceptance Rate')
plt.show()

In [ ]:
# Sample and trace plots
df = pd.DataFrame(states, columns=["x", "y"])

plt.figure(1, figsize=(8,8))
plt.subplot(121)
plt.scatter(df.x, df.y, c=target_dist.logpdf(states))
plt.axis('equal')
plt.title('Samples plot')
plt.xlabel('x')
plt.ylabel('y')

plt.subplot(222)
plt.plot(np.arange(states.shape[0]),df.x)
plt.title('Trace plot')
plt.xlabel('Steps')
plt.ylabel('x')

plt.subplot(224)
plt.scatter(np.arange(states.shape[0]),df.y)
plt.xlabel('Steps')
plt.ylabel('y')

plt.show()

In [ ]:
# All lags autocorrelation
autocorx = scipy.signal.correlate(states[:,0], states[:,0])
autocory = scipy.signal.correlate(states[:,1], states[:,1])

fig, (ax_autox, ax_autoy) = plt.subplots(2, 1, sharex=True, figsize=(7,7))

ax_autox.plot(range(-int(autocorx.size/2), int(autocorx.size/2) + 1), autocorx)
ax_autox.set_ylim(-1000,1000)
ax_autox.set_title('x')
ax_autox.set_ylabel('Autocorrelation')

ax_autoy.plot(range(-int(autocory.size/2), int(autocory.size/2) + 1), autocory)
# ax_autoy.set_ylim(-1000,1000)
ax_autoy.set_title('y')
ax_autoy.set_ylabel('Autocorrelation')
ax_autoy.set_xlabel('Lag')
plt.show()

In [ ]:
# Autocorrlation as a function of steps
def autocor_lagk_steps_1d(states, lag):
    autocor_lagk = []
    for i in range(len(states)):
        if i < lag:
            autocor_lagk.append(0)
        else:
            autocor = scipy.signal.correlate(states[:i], states[i:])
            autocor_lagk.append(autocor[int(len(autocor)/2) + lag])
    return np.asarray(autocor_lagk)

lag = 5
acx = autocor_lagk_steps_1d(states[:,0], lag)
acy = autocor_lagk_steps_1d(states[:,1], lag)

In [ ]:
# autocorrelation by steps plot
fig, (ax_autox, ax_autoy) = plt.subplots(2, 1, sharex=True, figsize=(7,7))

ax_autox.plot(range(len(states)), acx)
ax_autox.set_title('x')
ax_autox.set_ylabel('Autocorrelation')

ax_autoy.plot(range(len(states)), acy)
ax_autoy.set_title('y')
ax_autoy.set_ylabel('Autocorrelation')
ax_autoy.set_xlabel('Steps')

plt.show()